In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import spatialpy
from spatialpy import Model, Species, Parameter, Reaction, Mesh,\
                      PlaceInitialCondition, \
                      UniformInitialCondition, \
                      ScatterInitialCondition

# PolarizationInYeast

In [ ]:
class PolarizationInYeast(Model):
    def __init__(self):
        Model.__init__(self, name="PolarizationInYeast")

        # Domain
        mesh = Mesh.read_stochss_domain('PolarizationInYeast.smdl')
        self.add_mesh(mesh)

        self.staticDomain = True

        # Variables
        Cdc42c = Species(name="Cdc42c", diffusion_constant=10)
        Cdc42m = Species(name="Cdc42m", diffusion_constant=0.0053)
        self.add_species([Cdc42c, Cdc42m])

        self.restrict(Cdc42m, [2])

        # Initial Conditions
        self.add_initial_condition(ScatterInitialCondition(species=Cdc42c, count=300, types=[1]))
        self.add_initial_condition(ScatterInitialCondition(species=Cdc42m, count=100, types=[2]))

        # Parameters
        kon = Parameter(name="kon", expression="1.6666666667e-05")
        koff = Parameter(name="koff", expression="0.15")
        kfb = Parameter(name="kfb", expression="0.166666666666667")
        self.add_parameter([kon, koff, kfb])

        # Reactions
        r1 = Reaction(name="r1", reactants={Cdc42c: 1}, products={Cdc42m: 1}, rate=self.listOfParameters["kon"])
        r2 = Reaction(name="r2", reactants={Cdc42m: 1}, products={Cdc42c: 1}, rate=self.listOfParameters["koff"])
        r3 = Reaction(name="r3", restrict_to=[2], reactants={Cdc42c: 1, Cdc42m: 1}, products={Cdc42m: 2}, rate=self.listOfParameters["kfb"])
        self.add_reaction([r1, r2, r3])

        # Timespan
        self.timespan(np.arange(0, 5000, 10), timestep_size=10)

In [ ]:
model = PolarizationInYeast()

# Simulation Parameters

In [ ]:
def configure_simulation():
    kwargs = {
        # "number_of_trajectories":1,
        # "seed":None,
    }
    return kwargs

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

# Visualization

In [ ]:
results.plot_species('Cdc42c', animated=True, width=None, height=None)